# MNIST image classifier

딥러닝 예시에서 가장 많이 사용되는 데이터셋
![](https://www.tensorflow.org/images/MNIST.png)
- input: 28 x 28 image
- output: 0~9 numbers

#### references
- https://www.tensorflow.org/get_started/mnist/beginners
- https://www.tensorflow.org/get_started/mnist/pros
- https://github.com/tensorflow/tensorflow/blob/r1.1/tensorflow/examples/tutorials/mnist/mnist_softmax.py

# The basic model: softmax

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# Basic model parameters as external flags.
FLAGS = tf.app.flags.FLAGS
FLAGS.data_dir = './data'

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

# Import data
mnist = input_data.read_data_sets('./data', one_hot=True)

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz


In [4]:
# Create the model
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.matmul(x, W) + b

In [5]:
# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])

In [6]:
# The raw formulation of cross-entropy,
#
#   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
#                                 reduction_indices=[1]))
#
# can be numerically unstable.
#
# So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
# outputs of 'y', and then average across the batch.
cross_entropy = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [ ]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [ ]:
# Train
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [ ]:
# Test trained model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                  y_: mnist.test.labels}))

# More complicated models...

은닉 노드를 늘려보거나 층을 좀 더 쌓아보자.

아니면, convolutional & pooling layer를 도입해보자.

In [ ]:
# Create the model
x = tf.placeholder(tf.float32, [None, 784])
# [batch_size, image_width, image_height, channels]
x_reshaped = tf.reshape(x, [-1, 28, 28, 1])
# convolutional layer
conv = tf.contrib.layers.conv2d(
    inputs=x_reshaped,
    num_outputs=1, 
    kernel_size=[3, 3])
# pooling layer
pool = tf.contrib.layers.max_pool2d(
    inputs=conv,
    kernel_size=[2, 2],
    stride=2)
flat = tf.contrib.layers.flatten(pool)
flat = tf.contrib.layers.fully_connected(flat, 500)
y = tf.contrib.layers.linear(flat, 10)

In [ ]:
# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])
# The raw formulation of cross-entropy,
#
#   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
#                                 reduction_indices=[1]))
#
# can be numerically unstable.
#
# So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
# outputs of 'y', and then average across the batch.
cross_entropy = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [ ]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
# Train
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
# Test trained model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                  y_: mnist.test.labels}))

`mnist_softmax.py`를 수정해보자.

#### [참고] 텍스트 편집기가 필요하다면?
- 서브라임 텍스트 2 https://www.sublimetext.com/2
- VS Code https://code.visualstudio.com/download

### [참고] `tensorflow.contrib.layers`
layer에 대한 high-level wrapper 모음
- https://www.tensorflow.org/api_guides/python/contrib.layers#Higher_level_ops_for_building_neural_network_layers

#### without tf.contrib.layers

In [ ]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.matmul(x, W) + b

#### using tf.contrib.layers

In [ ]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.contrib.layers.linear(x, 10)

# Tensorboard: log and visualize your work

#### 로그 남기기
- `tf.summary` 사용
- 코드: `mnist_summary.py`

#### 로그 살펴보기
- 터미널에 `tensorboard --logdir=./logs` 입력